In [17]:
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from Data_for_bp import data_gather_from_files,run_strategy_optimised,run_strategy_eval,ladderize_absolute,convert_to_grid_binomial_data,indicator_prep,build_lot_sizing
import seaborn as sns

In [2]:
tick_data = data_gather_from_files('1 jan 2021','31 jan 2021')['EURUSD.mid']

Iteration No: 1 started. Evaluating function at random point.


AttributeError: module 'numpy' has no attribute 'int'.
`np.int` was a deprecated alias for the builtin `int`. To avoid this error in existing code, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

# Adam pytorch

In [21]:
import torch
from torch.autograd import Variable

def torch_run_strategy_optimised(tick_data, grid_sizing, lot_sizing, ladder_depth=10, ladder_function=ladderize_absolute, multiplier=1, indicator=False, lookback=200):

    grid_jumps, binomial_data = convert_to_grid_binomial_data(tick_data, grid_sizing, ladder_function, ladder_depth)  # This function needs to be converted too
    indicator_data = []
    if indicator:
        indicator_data = indicator_prep(grid_jumps, grid_sizing, lookback=lookback)  # This function needs to be converted too

    T = len(binomial_data)

    current_lots = torch.tensor(0.)
    previous_lots = torch.tensor(0.)
    R_PNL = torch.tensor(0.)
    U_PNL = torch.tensor(0.)
    PNL = torch.tensor(0.)
    position = torch.tensor(0.)
    position_sizing = build_lot_sizing(lot_sizing, binomial_data, multiplier=multiplier, indicator_data=indicator_data)  # This function needs to be converted too
    avg_price = torch.tensor(0.)

    max_position = torch.tensor(0.)
    min_U_PNL = torch.tensor(0.)

    for t in range(T):
        lots_in_order = - position_sizing[t] * binomial_data[t]
        previous_lots = current_lots.clone()
        current_lots = previous_lots + lots_in_order
        position = current_lots * grid_jumps[t]

        closing_position = (current_lots * lots_in_order <= 0) & (lots_in_order != 0)
        opening_or_adding_position = ~closing_position

        R_PNL -= torch.where(closing_position, (grid_jumps[t] - avg_price) * lots_in_order, torch.tensor(0.))
        
        # Update avg price for opening or adding to a position
        avg_price = torch.where(opening_or_adding_position & (current_lots != 0), 
                                (avg_price * previous_lots + grid_jumps[t] * lots_in_order) / current_lots,
                                avg_price)

        PNL += previous_lots * (grid_jumps[t] - grid_jumps[t-1])
        U_PNL = (grid_jumps[t] - avg_price) * current_lots

        PNL = torch.round(PNL, 4)
        U_PNL = torch.round(U_PNL, 4)
        R_PNL = torch.round(R_PNL, 4)

        min_U_PNL = torch.where(U_PNL < min_U_PNL, U_PNL, min_U_PNL)
        max_position = torch.where(torch.abs(position) > max_position, torch.abs(position), max_position)

    return PNL, min_U_PNL, max_position, R_PNL

def torch_objective(params, tick_data):
    G, n = params
    profit, U_PNL, max_position, R_PNL = torch_run_strategy_optimised(tick_data, G, n)
    return -R_PNL

def torch_constraint1(params, tick_data):
    G, n = params
    _, _, max_position, _ = torch_run_strategy_optimised(tick_data, G, n)
    return 10e6 - max_position

def torch_constraint2(params, tick_data):
    G, n = params
    _, U_PNL, _, _ = torch_run_strategy_optimised(tick_data, G, n)
    return U_PNL + 150e3

# Initialize parameters
params = Variable(torch.Tensor([0.005, 500000]), requires_grad=True)
optimizer = torch.optim.Adam([params], lr=0.001)  # You can adjust the learning rate

num_epochs = 2  # You can adjust the number of epochs

for epoch in range(num_epochs):
    optimizer.zero_grad()
    loss = torch_objective(params, tick_data)
    
    # Add constraints to the loss using a penalty method
    c1 = torch_constraint1(params, tick_data)
    c2 = torch_constraint2(params, tick_data)
    if c1 < 0:  # This means the constraint is violated
        loss += 1000 * torch.abs(c1)  # The factor 1000 is a penalty coefficient and can be adjusted
    if c2 < 0:
        loss += 1000 * torch.abs(c2)
    
    loss.backward()
    optimizer.step()

optimal_G, optimal_n = params.detach().numpy()


RuntimeError: Can't call numpy() on Tensor that requires grad. Use tensor.detach().numpy() instead.